In [1]:
# Gerekli kütüphaneler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import accuracy_score
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import datasets
from sklearn.model_selection import train_test_split
from sklearn.ensemble import BaggingClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import AdaBoostClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from transformers import AutoTokenizer, BertForMaskedLM
from sklearn.metrics import accuracy_score
from scipy.stats import randint
from sklearn.utils import shuffle

In [2]:
from rotation_forest import RotationForestClassifier, RotationTreeClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold

In [3]:
#veri setinin indirilmesi
from datasets import load_dataset

dataset = load_dataset('interpress_news_category_tr_lite')

C:\Users\havva\anaconda3\lib\site-packages\datasets\load.py:1429: FutureWarning: The repository for interpress_news_category_tr_lite contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/interpress_news_category_tr_lite
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [4]:
# Küçük harfe çevirme işlemini uygulayan bir fonksiyon
def preprocess_text(example):
    example['content'] = example['content'].replace("I", "ı").lower()
    return example

# Train setindeki her bir öğeye işlemi uygulamak
dataset['train'] = dataset['train'].map(preprocess_text)

# Test setindeki her bir öğeye işlemi uygulamak
dataset['test'] = dataset['test'].map(preprocess_text)

# Şimdi "content" sütunundaki metinler küçük harfe çevrilmiş oldular
print(dataset['train'][0]['content'])
print(dataset['test'][0]['content'])

tarihten sınıfta kaldık bugün tarihe damgasını vuran osmanlı i̇mparatorluğu nun kuruluş yıldönümü. adına dizilerin çekildiği tarihimizi ne kadar biliyoruz? gerekçeler faklı; ama sonuç aynı çıktı. tarihten sınıfta kaldık. sayfa 5r 1 bugün tarihe damgasını vuran osmanlı i̇mparatorluğumun kuruluş yıldönümü. adına dizilerin çekildiği tarihimizi ne kadar biliyoruz? gerekçeler faklı; ama sonuç aynı çıktı. tarihten sınıfta kaldık 7 ocak 1299... kıtalara dağılan ücüyle, ülkeler arasında gördüğü aygıyla tarihe damgasını vuran anlı devletin kuruluş tarihi. peki, anlı tarihimizi ne kadar biliyoruz? on zamanlarda tarihimizi anlatan izilere ilgi nasıl? bu dizilerde anlatanlar ne kadar sağlıklı? i̇şte sokaın değerlendirmesi; levlüdiye karaman (42-ev lamım):  bir bilgim yok. tarihle izla ilgilenmiyorum. eşim daha ilgilidir bu konuda. evde anlatır, ndan duyduklarımla yetiniyorum esem yalan olmaz. osmanlı döeminde yaşamak isterdim. tarih izileri izlerim muhteşem yüzyıl izisini çok izledim; hatta hiç ka

In [5]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['content', 'category'],
        num_rows: 218880
    })
    test: Dataset({
        features: ['content', 'category'],
        num_rows: 54721
    })
})


In [6]:
X_train = dataset['train']['content']
y_train = dataset['train']['category']

X_test = dataset['test']['content']
y_test = dataset['test']['category']

In [7]:
X_train_shuffled, y_train_shuffled = shuffle(X_train, y_train, random_state=42)
X_test_shuffled, y_test_shuffled = shuffle(X_test, y_test, random_state=42)

In [8]:
#eğitim için 5000 örnek ayrıldı
subset_size = 5000
X_train_sampled = X_train_shuffled[:subset_size]
y_train_sampled = y_train_shuffled[:subset_size]

In [9]:
#test için 500 örnek ayrıldı
subset_size = 500
X_test_sampled = X_test_shuffled[:subset_size]
y_test_sampled = y_test_shuffled[:subset_size]

In [10]:
print("X_train:", X_train_sampled[:3])  
print("y_train:", y_train_sampled[:3]) 

print("X_test:", X_test_sampled[:2])
print("y_test:", y_test_sampled[:2])

X_train: ['bayramda sahillere ineceklere 80 öneri emre tor,bayram için sahillerden 80 öneri bugüne kadar köşemde türkiye nin ve dünyanın farklı yerlerinden yüzlerce tavsiyede bulundum. şimdi sıra bayram önerilerinde. ege ve akdeniz sahilleri bu mevsimde hala çok güzel, gelin sizinle sahillerdeki tatil beldelerinde bulunan favori mekanlarımı paylaşayım böcek serin günlerden sıcak denizlere dunyagezer saffet emre stonguc@hurriyet.com.tr l-i̇da blue hotel  adatepe: huzurun adresi. 2-da costa hotel  assos: alabildiğine ege. 3-teomida hotel  burhaniye: dev bahçe. 4-ortunç hotel  ayvalık: eşsiz koy. 5-ayna restaurant  cunda: balığın en iyisi. 6-mola hotel  cunda: ev rahatlığı. 7-gulet restaurant  küçükkuyu: makul fiyat. 8-limonata hotel  assos: ferah. 9-cihat usta nın yeri  assos: balığın alası. 10-assosyal hotel: şaşırtıcı. ıı  ? m m ı 1-alavya hotel: şık. 2-alancha restaurant: lezzet mabedi. 3-langaza restaurant: sıradışı. 4-köyün delisi bar: çok keyifli. 5-traktör bar: ortam şahane. 6-vie

In [9]:
#büyük bert kullanımı
from transformers import AutoModel, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-turkish-uncased")
model = AutoModel.from_pretrained("dbmdz/bert-base-turkish-uncased")

In [10]:
#büyük bert için bert transformer oluşturulması
import torch

class BertTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        input_ids = []
        attention_masks = []

        for text in X:
            encoded_text = self.tokenizer.encode_plus(
                text,
                add_special_tokens=True,
                max_length=64,
                padding='max_length',
                truncation=True,
                return_tensors='pt'
            )

            input_ids.append(encoded_text['input_ids'])
            attention_masks.append(encoded_text['attention_mask'])

        input_ids = torch.cat(input_ids, dim=0)
        attention_masks = torch.cat(attention_masks, dim=0)

        with torch.no_grad():  # Gradyanları hesaplamadan ileri
            outputs = self.model(input_ids, attention_mask=attention_masks)
            pooled_output = outputs.pooler_output

        return pooled_output.numpy()


In [12]:
# ADABOOST

# AdaBoostClassifier için pipeline
pipeline_ab = Pipeline([
    ('bert', BertTransformer(model, tokenizer)),
    ('clf', AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=1), random_state=42))
])

# Hiperparametre aralıkları
param_grid = {
    'clf__n_estimators': [100],
    'clf__learning_rate': [0.1]
}

# GridSearchCV
grid_search_ab = GridSearchCV(pipeline_ab, param_grid, scoring='accuracy', cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42))

# Modeli eğitme ve en iyi parametreleri
grid_search_ab.fit(X_train_sampled, y_train_sampled)

# En iyi parametreleri ve sonuçları 
print("En iyi parametreler:", grid_search_ab.best_params_)
print("En iyi skor:", grid_search_ab.best_score_)

# Modelin performansını değerlendirin
y_pred_ab = grid_search_ab.predict(X_test_sampled)
print(classification_report(y_test_sampled, y_pred_ab))

# Her bir kombinasyonun performansı
results = grid_search_ab.cv_results_
for mean_score, params in zip(results['mean_test_score'], results['params']):
    print(f"Parametreler: {params}, Doğruluk: {mean_score * 100:.2f}%")

En iyi parametreler: {'clf__learning_rate': 0.1, 'clf__n_estimators': 100}
En iyi skor: 0.5116
              precision    recall  f1-score   support

           0       0.47      0.89      0.62       163
           1       0.42      0.63      0.50        82
           2       1.00      0.05      0.09        21
           3       0.00      0.00      0.00        33
           4       0.00      0.00      0.00        12
           5       0.96      0.31      0.47        71
           6       0.74      0.34      0.47        76
           7       0.00      0.00      0.00        12
           8       0.50      0.38      0.43        13
           9       0.00      0.00      0.00        17

    accuracy                           0.50       500
   macro avg       0.41      0.26      0.26       500
weighted avg       0.53      0.50      0.44       500

Parametreler: {'clf__learning_rate': 0.1, 'clf__n_estimators': 100}, Doğruluk: 51.16%


C:\Users\havva\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\havva\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\havva\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [13]:
#SVM

from sklearn.svm import SVC

# SVM için pipeline 
pipeline_svm = Pipeline([
    ('bert', BertTransformer(model, tokenizer)),
    ('clf', SVC())
])

# Hiperparametre aralıkları
param_grid = {
    'clf__C': [0.1],
    'clf__kernel': ['linear'],
}

# GridSearchCV
grid_search_svm = GridSearchCV(pipeline_svm, param_grid, scoring='accuracy', cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42))

# Modeli eğitme ve en iyi parametreler
grid_search_svm.fit(X_train_sampled, y_train_sampled)

# En iyi parametreleri ve sonuçları
print("En iyi parametreler:", grid_search_svm.best_params_)
print("En iyi skor:", grid_search_svm.best_score_)

# Modelin performansını değerlendirin
y_pred_svm = grid_search_svm.predict(X_test_sampled)
print(classification_report(y_test_sampled, y_pred_svm))

# Her bir kombinasyonun performansı
results = grid_search_svm.cv_results_
for mean_score, params in zip(results['mean_test_score'], results['params']):
    print(f"Parametreler: {params}, Doğruluk: {mean_score * 100:.2f}%")


En iyi parametreler: {'clf__C': 0.1, 'clf__kernel': 'linear'}
En iyi skor: 0.759
              precision    recall  f1-score   support

           0       0.73      0.82      0.77       163
           1       0.73      0.73      0.73        82
           2       0.55      0.52      0.54        21
           3       0.76      0.67      0.71        33
           4       0.62      0.42      0.50        12
           5       0.99      0.93      0.96        71
           6       0.78      0.78      0.78        76
           7       0.40      0.17      0.24        12
           8       0.50      0.62      0.55        13
           9       0.92      0.71      0.80        17

    accuracy                           0.76       500
   macro avg       0.70      0.64      0.66       500
weighted avg       0.76      0.76      0.75       500

Parametreler: {'clf__C': 0.1, 'clf__kernel': 'linear'}, Doğruluk: 75.90%


In [14]:
#RANDOM FOREST

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline

# Random Forest sınıflandırıcıyı ve BERT dönüşümünü kullanarak bir pipeline oluşturmak
pipeline_rf = Pipeline([
    ('bert', BertTransformer(model, tokenizer)),
    ('clf', RandomForestClassifier())  # Random Forest sınıflandırıcı burada
])

# Modelin hiperparametre optimizasyonu için bir ızgara araması (GridSearch) yapın (örn. parametreler)
param_grid_rf = {
    'clf__n_estimators': [150],  # ağaç sayısı
    'clf__max_depth': [30],  # Ağaç derinliği için farklı değerler
    'clf__min_samples_split': [10]   # Dallanma için gereken minimum örnek sayısı
}

grid_search_rf = GridSearchCV(pipeline_rf, param_grid_rf, cv=3, verbose=2)
grid_search_rf.fit(X_train_sampled, y_train_sampled)

# En iyi modelin ortalama doğruluğu
best_accuracy_rf = grid_search_rf.best_score_
print("En iyi ortalama doğruluk (accuracy) Random Forest:", best_accuracy_rf)

# Modelin performansı
y_pred_rf = grid_search_rf.predict(X_test_sampled)
print(classification_report(y_test_sampled, y_pred_rf))

# En iyi modelin parametreleri
print("En iyi parametreler Random Forest:", grid_search_rf.best_params_)


Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] END clf__max_depth=30, clf__min_samples_split=10, clf__n_estimators=150; total time= 6.4min
[CV] END clf__max_depth=30, clf__min_samples_split=10, clf__n_estimators=150; total time= 6.3min
[CV] END clf__max_depth=30, clf__min_samples_split=10, clf__n_estimators=150; total time= 6.3min
En iyi ortalama doğruluk (accuracy) Random Forest: 0.6340009309062556
              precision    recall  f1-score   support

           0       0.57      0.92      0.71       163
           1       0.56      0.74      0.64        82
           2       0.83      0.24      0.37        21
           3       1.00      0.06      0.11        33
           4       0.00      0.00      0.00        12
           5       1.00      0.75      0.85        71
           6       0.79      0.61      0.69        76
           7       0.00      0.00      0.00        12
           8       0.42      0.38      0.40        13
           9       0.00      0.00      

C:\Users\havva\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\havva\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\havva\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [16]:
# Bagging sınıflandırıcıyı ve BERT dönüşümünü kullanarak bir pipeline oluşturma
pipeline = Pipeline([
    ('bert', BertTransformer(model, tokenizer)),
    ('clf', BaggingClassifier())
])

# Modelin hiperparametre optimizasyonu için bir ızgara araması (GridSearch) yapın (örn. parametreler)
param_grid = {
    'clf__n_estimators': [100],
    'clf__max_samples': [0.5],
    'clf__max_features': [1.0]
}

grid_search = GridSearchCV(pipeline, param_grid, cv=3, verbose=2)
grid_search.fit(X_train_sampled, y_train_sampled)

# En iyi modelin ortalama doğruluğu
best_accuracy = grid_search.best_score_
print("En iyi ortalama doğruluk (accuracy):", best_accuracy)

# Modelin performansı
y_pred = grid_search.predict(X_test_sampled)
print(classification_report(y_test_sampled, y_pred))

# En iyi modelin parametreleri
print("En iyi parametreler:", grid_search.best_params_)

# Her bir kombinasyonun performansı
results = grid_search.cv_results_
for mean_score, params in zip(results['mean_test_score'], results['params']):
    print(f"Parametreler: {params}, Doğruluk: {mean_score * 100:.2f}%")


Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] END clf__max_features=1.0, clf__max_samples=0.5, clf__n_estimators=100; total time= 9.9min
[CV] END clf__max_features=1.0, clf__max_samples=0.5, clf__n_estimators=100; total time=10.5min
[CV] END clf__max_features=1.0, clf__max_samples=0.5, clf__n_estimators=100; total time=11.1min
En iyi ortalama doğruluk (accuracy): 0.6299988501699421
              precision    recall  f1-score   support

           0       0.60      0.84      0.70       163
           1       0.53      0.74      0.62        82
           2       0.54      0.33      0.41        21
           3       1.00      0.12      0.22        33
           4       1.00      0.17      0.29        12
           5       0.93      0.75      0.83        71
           6       0.73      0.63      0.68        76
           7       0.00      0.00      0.00        12
           8       0.38      0.38      0.38        13
           9       1.00      0.06      0.11        17

 

In [11]:
# Bagging sınıflandırıcıyı ve BERT dönüşümünü kullanarak bir pipeline oluşturma
pipeline = Pipeline([
    ('bert', BertTransformer(model, tokenizer)),
    ('clf', BaggingClassifier())
])

# Modelin hiperparametre optimizasyonu için bir ızgara araması (GridSearch) yapın (örn. parametreler)
param_grid = {
    'clf__n_estimators': [10, 50, 100],
    'clf__max_samples': [0.5, 1.0, 1.5],
    'clf__max_features': [0.5, 1.0, 1.5]
}

grid_search = GridSearchCV(pipeline, param_grid, cv=3, verbose=2)
grid_search.fit(X_train_sampled, y_train_sampled)

# En iyi modelin ortalama doğruluğu
best_accuracy = grid_search.best_score_
print("En iyi ortalama doğruluk (accuracy):", best_accuracy)

# Modelin performansı
y_pred = grid_search.predict(X_test_sampled)
print(classification_report(y_test_sampled, y_pred))

# En iyi modelin parametreleri
print("En iyi parametreler:", grid_search.best_params_)


Fitting 3 folds for each of 27 candidates, totalling 81 fits


C:\Users\havva\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


[CV] END clf__max_features=0.5, clf__max_samples=0.5, clf__n_estimators=10; total time=  10.9s
[CV] END clf__max_features=0.5, clf__max_samples=0.5, clf__n_estimators=10; total time=  10.7s
[CV] END clf__max_features=0.5, clf__max_samples=0.5, clf__n_estimators=10; total time=  11.6s
[CV] END clf__max_features=0.5, clf__max_samples=0.5, clf__n_estimators=50; total time=  11.2s
[CV] END clf__max_features=0.5, clf__max_samples=0.5, clf__n_estimators=50; total time=  11.8s
[CV] END clf__max_features=0.5, clf__max_samples=0.5, clf__n_estimators=50; total time=  12.9s
[CV] END clf__max_features=0.5, clf__max_samples=0.5, clf__n_estimators=100; total time=  12.3s
[CV] END clf__max_features=0.5, clf__max_samples=0.5, clf__n_estimators=100; total time=  11.9s
[CV] END clf__max_features=0.5, clf__max_samples=0.5, clf__n_estimators=100; total time=  11.6s
[CV] END clf__max_features=0.5, clf__max_samples=1.0, clf__n_estimators=10; total time=  10.6s
[CV] END clf__max_features=0.5, clf__max_sample

C:\Users\havva\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
45 fits failed out of a total of 81.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
18 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\havva\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\havva\anaconda3\lib\site-packages\sklearn\pipeline.py", line 405, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "C:\Users\havva\anaconda3\lib\site-packages\sklearn\ensemble\_bagging.py", line 326, in fit
    self._validate_params()
  File "C:\Users\havva\anaconda3

              precision    recall  f1-score   support

           0       0.60      0.75      0.67         4
           1       0.00      0.00      0.00         1
           3       0.00      0.00      0.00         1
           5       1.00      1.00      1.00         2
           6       1.00      0.50      0.67         2
           8       0.00      0.00      0.00         0

    accuracy                           0.60        10
   macro avg       0.43      0.38      0.39        10
weighted avg       0.64      0.60      0.60        10

En iyi parametreler: {'clf__max_features': 1.0, 'clf__max_samples': 0.5, 'clf__n_estimators': 100}


C:\Users\havva\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\havva\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\havva\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\havva\anaconda3\lib\site-packages\sklearn\metrics\_cla

In [11]:
#rotation forest
from sktime.classification.sklearn import RotationForest
rotation_tree_clf = RotationTreeClassifier()

# rotation Forest sınıflandırıcıyı ve BERT dönüşümünü kullanarak bir pipeline oluşturma
pipeline_rotation_forest = Pipeline([
    ('bert', BertTransformer(model, tokenizer)),
    ('clf',  RotationForest())
])

param_grid_rotation_forest = {
    'clf__n_estimators': [100, 150],
    'clf__min_group': [3, 5],
    'clf__max_group': [3, 5],
    'clf__remove_proportion': [0.5, 0.7],
    'clf__n_jobs': [-1],
}


grid_search_rf = GridSearchCV(pipeline_rotation_forest, param_grid_rotation_forest,scoring='accuracy', cv=3, verbose=2)
grid_search_rf.fit(X_train_sampled, y_train_sampled)

# En iyi modelin ortalama doğruluğu
best_accuracy_rf = grid_search_rf.best_score_
print("En iyi ortalama doğruluk (accuracy) Rotation Forest:", best_accuracy_rf)

# Modelin performansı
y_pred_rf = grid_search_rf.predict(X_test_sampled)
print(classification_report(y_test_sampled, y_pred_rf))

# En iyi modelin parametreleri
print("En iyi parametreler Rotation Forest:", grid_search_rf.best_params_)


Fitting 3 folds for each of 16 candidates, totalling 48 fits
[CV] END clf__max_group=3, clf__min_group=3, clf__n_estimators=100, clf__n_jobs=-1, clf__remove_proportion=0.5; total time=18.4min
[CV] END clf__max_group=3, clf__min_group=3, clf__n_estimators=100, clf__n_jobs=-1, clf__remove_proportion=0.5; total time=17.0min
[CV] END clf__max_group=3, clf__min_group=3, clf__n_estimators=100, clf__n_jobs=-1, clf__remove_proportion=0.5; total time=14.6min
[CV] END clf__max_group=3, clf__min_group=3, clf__n_estimators=100, clf__n_jobs=-1, clf__remove_proportion=0.7; total time=14.8min
[CV] END clf__max_group=3, clf__min_group=3, clf__n_estimators=100, clf__n_jobs=-1, clf__remove_proportion=0.7; total time=14.7min
[CV] END clf__max_group=3, clf__min_group=3, clf__n_estimators=100, clf__n_jobs=-1, clf__remove_proportion=0.7; total time=14.9min
[CV] END clf__max_group=3, clf__min_group=3, clf__n_estimators=150, clf__n_jobs=-1, clf__remove_proportion=0.5; total time=26.1min
[CV] END clf__max_grou

C:\Users\havva\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
12 fits failed out of a total of 48.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
12 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\havva\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\havva\anaconda3\lib\site-packages\sklearn\pipeline.py", line 405, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "C:\Users\havva\anaconda3\lib\site-packages\sktime\classification\sklearn\_rotation_forest.py", line 241, in fit
    fit = Parallel(n_jobs=self._n_jobs)

En iyi ortalama doğruluk (accuracy) Rotation Forest: 0.6706004537387841
              precision    recall  f1-score   support

           0       0.65      0.86      0.74       163
           1       0.61      0.80      0.69        82
           2       0.69      0.52      0.59        21
           3       0.90      0.27      0.42        33
           4       1.00      0.33      0.50        12
           5       0.95      0.82      0.88        71
           6       0.81      0.68      0.74        76
           7       1.00      0.08      0.15        12
           8       0.47      0.54      0.50        13
           9       1.00      0.29      0.45        17

    accuracy                           0.71       500
   macro avg       0.81      0.52      0.57       500
weighted avg       0.75      0.71      0.69       500

En iyi parametreler Rotation Forest: {'clf__max_group': 5, 'clf__min_group': 5, 'clf__n_estimators': 150, 'clf__n_jobs': -1, 'clf__remove_proportion': 0.5}


In [15]:
from transformers import AutoModel, AutoTokenizer
#mini bert kullanımı 
model = AutoModel.from_pretrained("ytu-ce-cosmos/turkish-mini-bert-uncased")
tokenizer = AutoTokenizer.from_pretrained("ytu-ce-cosmos/turkish-mini-bert-uncased")

In [23]:
import torch
from sklearn.base import BaseEstimator, TransformerMixin
from transformers import AutoTokenizer, BertForMaskedLM

class BertTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, model, tokenizer, max_length=64, batch_size=8):
        self.model = model
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.batch_size = batch_size

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        input_ids = []
        attention_masks = []

        for i in range(0, len(X), self.batch_size):
            batch_texts = X[i:i+self.batch_size]
            batch_encoded_texts = self.tokenizer.batch_encode_plus(
                batch_texts,
                add_special_tokens=True,
                max_length=self.max_length,
                padding='max_length',
                truncation=True,
                return_tensors='pt'
            )

            input_ids.append(batch_encoded_texts['input_ids'])
            attention_masks.append(batch_encoded_texts['attention_mask'])

        input_ids = torch.cat(input_ids, dim=0)
        attention_masks = torch.cat(attention_masks, dim=0)

        with torch.no_grad():
            outputs = self.model(input_ids, attention_mask=attention_masks)
            pooled_output = outputs.last_hidden_state.mean(dim=1)  # Ortalama al
            # pooled_output = outputs.pooler_output  # Bu satırı kullanmak yerine ortalama almayı tercih ettim

        return pooled_output.numpy()


In [24]:
import torch
from torch.utils.data import DataLoader, TensorDataset

# Bagging sınıflandırıcıyı ve BERT dönüşümünü kullanarak bir pipeline oluşturma
pipeline = Pipeline([
    ('bert', BertTransformer(model, tokenizer)),
    ('clf', BaggingClassifier())
])

# Modelin hiperparametre optimizasyonu için bir ızgara araması (GridSearch) yapın (örn. parametreler)
param_grid = {
    'clf__n_estimators': [100],
    'clf__max_samples': [0.5],
    'clf__max_features': [1.0]
}

grid_search = GridSearchCV(pipeline, param_grid, scoring='accuracy', cv=3, verbose=2)
grid_search.fit(X_train_sampled, y_train_sampled)

# En iyi modelin ortalama doğruluğunu alma
best_accuracy = grid_search.best_score_
print("En iyi ortalama doğruluk (accuracy):", best_accuracy)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] END clf__max_features=1.0, clf__max_samples=0.5, clf__n_estimators=100; total time= 1.7min
[CV] END clf__max_features=1.0, clf__max_samples=0.5, clf__n_estimators=100; total time= 1.7min
[CV] END clf__max_features=1.0, clf__max_samples=0.5, clf__n_estimators=100; total time= 1.7min
En iyi ortalama doğruluk (accuracy): 0.6791981339626432


In [25]:
y_pred = grid_search.predict(X_test_sampled)
print(classification_report(y_test_sampled, y_pred))

# En iyi modelin parametrelerini görüntüleme
print("En iyi parametreler:", grid_search.best_params_)

# Her bir kombinasyonun performansı
results = grid_search.cv_results_
for mean_score, params in zip(results['mean_test_score'], results['params']):
    print(f"Parametreler: {params}, Doğruluk: {mean_score * 100:.2f}%")


              precision    recall  f1-score   support

           0       0.60      0.86      0.71       163
           1       0.63      0.78      0.70        82
           2       0.80      0.38      0.52        21
           3       1.00      0.18      0.31        33
           4       0.00      0.00      0.00        12
           5       0.89      0.80      0.84        71
           6       0.76      0.66      0.70        76
           7       0.00      0.00      0.00        12
           8       0.44      0.54      0.48        13
           9       1.00      0.06      0.11        17

    accuracy                           0.67       500
   macro avg       0.61      0.43      0.44       500
weighted avg       0.68      0.67      0.63       500

En iyi parametreler: {'clf__max_features': 1.0, 'clf__max_samples': 0.5, 'clf__n_estimators': 100}
Parametreler: {'clf__max_features': 1.0, 'clf__max_samples': 0.5, 'clf__n_estimators': 100}, Doğruluk: 67.92%


C:\Users\havva\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\havva\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\havva\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [26]:
#adaboost

pipeline_ab = Pipeline([
    ('bert', BertTransformer(model, tokenizer)),
    ('clf', AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1), random_state=42))
])

param_grid = {
    'clf__n_estimators': [100],
    'clf__learning_rate': [0.1]
}


grid_search_ab = GridSearchCV(pipeline_ab, param_grid, scoring='accuracy', cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42))


grid_search_ab.fit(X_train_sampled, y_train_sampled)


print("Best parameters:", grid_search_ab.best_params_)
print("Best score:", grid_search_ab.best_score_)

y_pred = grid_search.predict(X_test_sampled)
print(classification_report(y_test_sampled, y_pred))


results = grid_search_ab.cv_results_
for mean_score, params in zip(results['mean_test_score'], results['params']):
    print(f"Parameters: {params}, Accuracy: {mean_score * 100:.2f}%")

C:\Users\havva\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\havva\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\havva\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\havva\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\havva\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\havva\anaconda3\lib\site-p

Best parameters: {'clf__learning_rate': 0.1, 'clf__n_estimators': 100}
Best score: 0.4822
              precision    recall  f1-score   support

           0       0.60      0.86      0.71       163
           1       0.63      0.78      0.70        82
           2       0.80      0.38      0.52        21
           3       1.00      0.18      0.31        33
           4       0.00      0.00      0.00        12
           5       0.89      0.80      0.84        71
           6       0.76      0.66      0.70        76
           7       0.00      0.00      0.00        12
           8       0.44      0.54      0.48        13
           9       1.00      0.06      0.11        17

    accuracy                           0.67       500
   macro avg       0.61      0.43      0.44       500
weighted avg       0.68      0.67      0.63       500

Parameters: {'clf__learning_rate': 0.1, 'clf__n_estimators': 100}, Accuracy: 48.22%


C:\Users\havva\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\havva\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\havva\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [27]:
#SVM

from sklearn.svm import SVC

# SVM için pipeline 
pipeline_svm = Pipeline([
    ('bert', BertTransformer(model, tokenizer)),
    ('clf', SVC())
])

# Hiperparametre aralıkları
param_grid = {
    'clf__C': [0.1],
    'clf__kernel': ['linear'],
}

# GridSearchCVç
grid_search_svm = GridSearchCV(pipeline_svm, param_grid, scoring='accuracy', cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42))

# Modeli eğitme ve en iyi parametreler
grid_search_svm.fit(X_train_sampled, y_train_sampled)

# En iyi parametreleri ve sonuçları
print("En iyi parametreler:", grid_search_svm.best_params_)
print("En iyi skor:", grid_search_svm.best_score_)

# Modelin performansı
y_pred_svm = grid_search_svm.predict(X_test_sampled)
print(classification_report(y_test_sampled, y_pred_svm))

# Her bir kombinasyonun performansı
results = grid_search_svm.cv_results_
for mean_score, params in zip(results['mean_test_score'], results['params']):
    print(f"Parametreler: {params}, Doğruluk: {mean_score * 100:.2f}%")


En iyi parametreler: {'clf__C': 0.1, 'clf__kernel': 'linear'}
En iyi skor: 0.7736000000000001
              precision    recall  f1-score   support

           0       0.75      0.79      0.77       163
           1       0.76      0.80      0.78        82
           2       0.75      0.57      0.65        21
           3       0.72      0.64      0.68        33
           4       0.75      0.50      0.60        12
           5       0.94      0.94      0.94        71
           6       0.79      0.80      0.80        76
           7       0.71      0.42      0.53        12
           8       0.39      0.69      0.50        13
           9       0.91      0.59      0.71        17

    accuracy                           0.77       500
   macro avg       0.75      0.67      0.70       500
weighted avg       0.78      0.77      0.77       500

Parametreler: {'clf__C': 0.1, 'clf__kernel': 'linear'}, Doğruluk: 77.36%


In [28]:
#RANDOM FOREST

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline

pipeline_rf = Pipeline([
    ('bert', BertTransformer(model, tokenizer)),
    ('clf', RandomForestClassifier())  # Random Forest sınıflandırıcı burada
])

param_grid_rf = {
    'clf__n_estimators': [150], 
    'clf__max_depth': [30],  
    'clf__min_samples_split': [10]    
}

grid_search_rf = GridSearchCV(pipeline_rf, param_grid_rf, cv=3, verbose=2)
grid_search_rf.fit(X_train_sampled, y_train_sampled)

best_accuracy_rf = grid_search_rf.best_score_
print("En iyi ortalama doğruluk (accuracy) Random Forest:", best_accuracy_rf)

y_pred_rf = grid_search_rf.predict(X_test_sampled)
print(classification_report(y_test_sampled, y_pred_rf))

print("En iyi parametreler Random Forest:", grid_search_rf.best_params_)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] END clf__max_depth=30, clf__min_samples_split=10, clf__n_estimators=150; total time=  37.3s
[CV] END clf__max_depth=30, clf__min_samples_split=10, clf__n_estimators=150; total time=  37.2s
[CV] END clf__max_depth=30, clf__min_samples_split=10, clf__n_estimators=150; total time=  37.2s
En iyi ortalama doğruluk (accuracy) Random Forest: 0.6865990547388722
              precision    recall  f1-score   support

           0       0.63      0.92      0.75       163
           1       0.65      0.80      0.72        82
           2       1.00      0.43      0.60        21
           3       1.00      0.21      0.35        33
           4       1.00      0.08      0.15        12
           5       0.98      0.83      0.90        71
           6       0.79      0.71      0.75        76
           7       0.00      0.00      0.00        12
           8       0.47      0.54      0.50        13
           9       0.00      0.00      

C:\Users\havva\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\havva\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\havva\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
#rotation forest
from sktime.classification.sklearn import RotationForest
rotation_tree_clf = RotationTreeClassifier()

pipeline_rotation_forest = Pipeline([
    ('bert', BertTransformer(model, tokenizer)),
    ('clf',  RotationForest())
])

param_grid_rotation_forest = {
    'clf__n_estimators': [150],
    'clf__min_group': [5],
    'clf__max_group': [5],
    'clf__remove_proportion': [0.5],
    'clf__n_jobs': [-1],
}


grid_search_rf = GridSearchCV(pipeline_rotation_forest, param_grid_rotation_forest,scoring='accuracy', cv=3, verbose=2)
grid_search_rf.fit(X_train_sampled, y_train_sampled)

best_accuracy_rf = grid_search_rf.best_score_
print("En iyi ortalama doğruluk (accuracy) Rotation Forest:", best_accuracy_rf)

y_pred_rf = grid_search_rf.predict(X_test_sampled)
print(classification_report(y_test_sampled, y_pred_rf))

print("En iyi parametreler Rotation Forest:", grid_search_rf.best_params_)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] END clf__max_group=5, clf__min_group=5, clf__n_estimators=150, clf__n_jobs=-1, clf__remove_proportion=0.5; total time= 5.5min
[CV] END clf__max_group=5, clf__min_group=5, clf__n_estimators=150, clf__n_jobs=-1, clf__remove_proportion=0.5; total time= 5.3min
[CV] END clf__max_group=5, clf__min_group=5, clf__n_estimators=150, clf__n_jobs=-1, clf__remove_proportion=0.5; total time= 5.4min
En iyi ortalama doğruluk (accuracy) Rotation Forest: 0.7196001375955302


In [16]:
#SMALL BERT
 
from transformers import AutoModel, AutoTokenizer

model = AutoModel.from_pretrained("ytu-ce-cosmos/turkish-small-bert-uncased")
tokenizer = AutoTokenizer.from_pretrained("ytu-ce-cosmos/turkish-small-bert-uncased")

In [17]:
import torch
from sklearn.base import BaseEstimator, TransformerMixin
from transformers import AutoTokenizer, BertForMaskedLM

class BertTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, model, tokenizer, max_length=64, batch_size=8):
        self.model = model
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.batch_size = batch_size

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        input_ids = []
        attention_masks = []

        for i in range(0, len(X), self.batch_size):
            batch_texts = X[i:i+self.batch_size]
            batch_encoded_texts = self.tokenizer.batch_encode_plus(
                batch_texts,
                add_special_tokens=True,
                max_length=self.max_length,
                padding='max_length',
                truncation=True,
                return_tensors='pt'
            )

            input_ids.append(batch_encoded_texts['input_ids'])
            attention_masks.append(batch_encoded_texts['attention_mask'])

        input_ids = torch.cat(input_ids, dim=0)
        attention_masks = torch.cat(attention_masks, dim=0)

        with torch.no_grad():
            outputs = self.model(input_ids, attention_mask=attention_masks)
            pooled_output = outputs.last_hidden_state.mean(dim=1)  # Ortalama al
            # pooled_output = outputs.pooler_output  # Bu satırı kullanmak yerine ortalama almayı tercih ettim

        return pooled_output.numpy()


In [14]:
import torch
from torch.utils.data import DataLoader, TensorDataset

pipeline = Pipeline([
    ('bert', BertTransformer(model, tokenizer)),
    ('clf', BaggingClassifier())
])

param_grid = {
    'clf__n_estimators': [100],
    'clf__max_samples': [0.5],
    'clf__max_features': [1.0]
}

grid_search = GridSearchCV(pipeline, param_grid, scoring='accuracy', cv=3, verbose=2)
grid_search.fit(X_train_sampled, y_train_sampled)

best_accuracy = grid_search.best_score_
print("En iyi ortalama doğruluk (accuracy):", best_accuracy)

y_pred = grid_search.predict(X_test_sampled)
print(classification_report(y_test_sampled, y_pred))

print("En iyi parametreler:", grid_search.best_params_)

results = grid_search.cv_results_
for mean_score, params in zip(results['mean_test_score'], results['params']):
    print(f"Parametreler: {params}, Doğruluk: {mean_score * 100:.2f}%")

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] END clf__max_features=1.0, clf__max_samples=0.5, clf__n_estimators=100; total time= 3.3min
[CV] END clf__max_features=1.0, clf__max_samples=0.5, clf__n_estimators=100; total time= 2.7min
[CV] END clf__max_features=1.0, clf__max_samples=0.5, clf__n_estimators=100; total time= 2.7min
En iyi ortalama doğruluk (accuracy): 0.6779981338666241
              precision    recall  f1-score   support

           0       0.63      0.90      0.74       163
           1       0.67      0.85      0.75        82
           2       0.78      0.33      0.47        21
           3       0.82      0.27      0.41        33
           4       1.00      0.17      0.29        12
           5       1.00      0.83      0.91        71
           6       0.79      0.74      0.76        76
           7       0.00      0.00      0.00        12
           8       0.42      0.38      0.40        13
           9       1.00      0.06      0.11        17

 

C:\Users\havva\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\havva\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\havva\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [18]:
#adaboost

pipeline_ab = Pipeline([
    ('bert', BertTransformer(model, tokenizer)),
    ('clf', AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1), random_state=42))
])

param_grid = {
    'clf__n_estimators': [100],
    'clf__learning_rate': [0.1]
}

grid_search_ab = GridSearchCV(pipeline_ab, param_grid, scoring='accuracy', cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42))

grid_search_ab.fit(X_train_sampled, y_train_sampled)

print("Best parameters:", grid_search_ab.best_params_)
print("Best score:", grid_search_ab.best_score_)

y_pred = grid_search_ab.predict(X_test_sampled)
print(classification_report(y_test_sampled, y_pred))

results = grid_search_ab.cv_results_
for mean_score, params in zip(results['mean_test_score'], results['params']):
    print(f"Parameters: {params}, Accuracy: {mean_score * 100:.2f}%")

C:\Users\havva\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\havva\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\havva\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\havva\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\havva\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\havva\anaconda3\lib\site-p

Best parameters: {'clf__learning_rate': 0.1, 'clf__n_estimators': 100}
Best score: 0.502
              precision    recall  f1-score   support

           0       0.47      0.92      0.63       163
           1       0.48      0.62      0.54        82
           2       1.00      0.10      0.17        21
           3       0.00      0.00      0.00        33
           4       0.00      0.00      0.00        12
           5       1.00      0.49      0.66        71
           6       0.75      0.39      0.52        76
           7       0.00      0.00      0.00        12
           8       1.00      0.08      0.14        13
           9       0.00      0.00      0.00        17

    accuracy                           0.54       500
   macro avg       0.47      0.26      0.27       500
weighted avg       0.56      0.54      0.48       500

Parameters: {'clf__learning_rate': 0.1, 'clf__n_estimators': 100}, Accuracy: 50.20%


C:\Users\havva\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\havva\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\havva\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [16]:
#SVM

from sklearn.svm import SVC

# SVM için pipeline 
pipeline_svm = Pipeline([
    ('bert', BertTransformer(model, tokenizer)),
    ('clf', SVC())
])

# Hiperparametre aralıkları
param_grid = {
    'clf__C': [0.1],
    'clf__kernel': ['linear'],
}

# GridSearchCVç
grid_search_svm = GridSearchCV(pipeline_svm, param_grid, scoring='accuracy', cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42))

# Modeli eğitme ve en iyi parametreler
grid_search_svm.fit(X_train_sampled, y_train_sampled)

# En iyi parametreleri ve sonuçları
print("En iyi parametreler:", grid_search_svm.best_params_)
print("En iyi skor:", grid_search_svm.best_score_)

# Modelin performansıı
y_pred_svm = grid_search_svm.predict(X_test_sampled)
print(classification_report(y_test_sampled, y_pred_svm))

# Her bir kombinasyonun performansı
results = grid_search_svm.cv_results_
for mean_score, params in zip(results['mean_test_score'], results['params']):
    print(f"Parametreler: {params}, Doğruluk: {mean_score * 100:.2f}%")


En iyi parametreler: {'clf__C': 0.1, 'clf__kernel': 'linear'}
En iyi skor: 0.7924
              precision    recall  f1-score   support

           0       0.78      0.80      0.79       163
           1       0.80      0.79      0.80        82
           2       0.67      0.57      0.62        21
           3       0.79      0.82      0.81        33
           4       0.50      0.50      0.50        12
           5       0.99      0.93      0.96        71
           6       0.78      0.84      0.81        76
           7       0.50      0.33      0.40        12
           8       0.44      0.62      0.52        13
           9       0.93      0.76      0.84        17

    accuracy                           0.79       500
   macro avg       0.72      0.70      0.70       500
weighted avg       0.79      0.79      0.79       500

Parametreler: {'clf__C': 0.1, 'clf__kernel': 'linear'}, Doğruluk: 79.24%


In [17]:
#RANDOM FOREST

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline

pipeline_rf = Pipeline([
    ('bert', BertTransformer(model, tokenizer)),
    ('clf', RandomForestClassifier())  # Random Forest sınıflandırıcı burada
])

param_grid_rf = {
    'clf__n_estimators': [150],
    'clf__max_depth': [30],  
    'clf__min_samples_split': [10]  
}

grid_search_rf = GridSearchCV(pipeline_rf, param_grid_rf, cv=3, verbose=2)
grid_search_rf.fit(X_train_sampled, y_train_sampled)


best_accuracy_rf = grid_search_rf.best_score_
print("En iyi ortalama doğruluk (accuracy) Random Forest:", best_accuracy_rf)


y_pred_rf = grid_search_rf.predict(X_test_sampled)
print(classification_report(y_test_sampled, y_pred_rf))


print("En iyi parametreler Random Forest:", grid_search_rf.best_params_)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] END clf__max_depth=30, clf__min_samples_split=10, clf__n_estimators=150; total time= 1.0min
[CV] END clf__max_depth=30, clf__min_samples_split=10, clf__n_estimators=150; total time= 1.0min
[CV] END clf__max_depth=30, clf__min_samples_split=10, clf__n_estimators=150; total time= 1.1min
En iyi ortalama doğruluk (accuracy) Random Forest: 0.6759994939787553
              precision    recall  f1-score   support

           0       0.60      0.91      0.72       163
           1       0.61      0.76      0.67        82
           2       1.00      0.33      0.50        21
           3       1.00      0.09      0.17        33
           4       0.00      0.00      0.00        12
           5       1.00      0.86      0.92        71
           6       0.79      0.72      0.75        76
           7       0.00      0.00      0.00        12
           8       0.50      0.38      0.43        13
           9       1.00      0.06      

C:\Users\havva\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\havva\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\havva\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [12]:
#rotation forest
from sktime.classification.sklearn import RotationForest
rotation_tree_clf = RotationTreeClassifier()


pipeline_rotation_forest = Pipeline([
    ('bert', BertTransformer(model, tokenizer)),
    ('clf',  RotationForest())
])

param_grid_rotation_forest = {
    'clf__n_estimators': [150],
    'clf__min_group': [5],
    'clf__max_group': [5],
    'clf__remove_proportion': [0.5],
    'clf__n_jobs': [-1],
}


grid_search_rf = GridSearchCV(pipeline_rotation_forest, param_grid_rotation_forest,scoring='accuracy', cv=3, verbose=2)
grid_search_rf.fit(X_train_sampled, y_train_sampled)


best_accuracy_rf = grid_search_rf.best_score_
print("En iyi ortalama doğruluk (accuracy) Rotation Forest:", best_accuracy_rf)


y_pred_rf = grid_search_rf.predict(X_test_sampled)
print(classification_report(y_test_sampled, y_pred_rf))


print("En iyi parametreler Rotation Forest:", grid_search_rf.best_params_)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] END clf__max_group=5, clf__min_group=5, clf__n_estimators=150, clf__n_jobs=-1, clf__remove_proportion=0.5; total time=19.5min
[CV] END clf__max_group=5, clf__min_group=5, clf__n_estimators=150, clf__n_jobs=-1, clf__remove_proportion=0.5; total time=16.0min
[CV] END clf__max_group=5, clf__min_group=5, clf__n_estimators=150, clf__n_jobs=-1, clf__remove_proportion=0.5; total time=10.7min
En iyi ortalama doğruluk (accuracy) Rotation Forest: 0.7262002581476502
              precision    recall  f1-score   support

           0       0.67      0.90      0.77       163
           1       0.71      0.83      0.76        82
           2       0.75      0.43      0.55        21
           3       0.72      0.39      0.51        33
           4       0.67      0.17      0.27        12
           5       1.00      0.90      0.95        71
           6       0.87      0.76      0.81        76
           7       0.50      0.08      0.14

In [10]:
#TINY BERT
 
from transformers import AutoModel, AutoTokenizer

model = AutoModel.from_pretrained("ytu-ce-cosmos/turkish-tiny-bert-uncased")
tokenizer = AutoTokenizer.from_pretrained("ytu-ce-cosmos/turkish-tiny-bert-uncased")

In [11]:
import torch
from sklearn.base import BaseEstimator, TransformerMixin
from transformers import AutoTokenizer, BertForMaskedLM

class BertTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, model, tokenizer, max_length=64, batch_size=8):
        self.model = model
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.batch_size = batch_size

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        input_ids = []
        attention_masks = []

        for i in range(0, len(X), self.batch_size):
            batch_texts = X[i:i+self.batch_size]
            batch_encoded_texts = self.tokenizer.batch_encode_plus(
                batch_texts,
                add_special_tokens=True,
                max_length=self.max_length,
                padding='max_length',
                truncation=True,
                return_tensors='pt'
            )

            input_ids.append(batch_encoded_texts['input_ids'])
            attention_masks.append(batch_encoded_texts['attention_mask'])

        input_ids = torch.cat(input_ids, dim=0)
        attention_masks = torch.cat(attention_masks, dim=0)

        with torch.no_grad():
            outputs = self.model(input_ids, attention_mask=attention_masks)
            pooled_output = outputs.last_hidden_state.mean(dim=1)  # Ortalama al
            # pooled_output = outputs.pooler_output  # Bu satırı kullanmak yerine ortalama almayı tercih ettim

        return pooled_output.numpy()

In [17]:
#adaboost

pipeline_ab = Pipeline([
    ('bert', BertTransformer(model, tokenizer)),
    ('clf', AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1), random_state=42))
])


param_grid = {
    'clf__n_estimators': [100],
    'clf__learning_rate': [0.1]
}


grid_search_ab = GridSearchCV(pipeline_ab, param_grid, scoring='accuracy', cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42))


grid_search_ab.fit(X_train_sampled, y_train_sampled)


print("Best parameters:", grid_search_ab.best_params_)
print("Best score:", grid_search_ab.best_score_)

y_pred = grid_search_ab.predict(X_test_sampled)
print(classification_report(y_test_sampled, y_pred))

results = grid_search_ab.cv_results_
for mean_score, params in zip(results['mean_test_score'], results['params']):
    print(f"Parameters: {params}, Accuracy: {mean_score * 100:.2f}%")

C:\Users\havva\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\havva\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\havva\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\havva\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\havva\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\havva\anaconda3\lib\site-p

Best parameters: {'clf__learning_rate': 0.1, 'clf__n_estimators': 100}
Best score: 0.5471999999999999
              precision    recall  f1-score   support

           0       0.49      0.94      0.65       163
           1       0.55      0.59      0.56        82
           2       0.00      0.00      0.00        21
           3       1.00      0.03      0.06        33
           4       0.00      0.00      0.00        12
           5       0.98      0.63      0.77        71
           6       0.79      0.50      0.61        76
           7       0.00      0.00      0.00        12
           8       0.25      0.08      0.12        13
           9       0.00      0.00      0.00        17

    accuracy                           0.57       500
   macro avg       0.41      0.28      0.28       500
weighted avg       0.58      0.57      0.51       500

Parameters: {'clf__learning_rate': 0.1, 'clf__n_estimators': 100}, Accuracy: 54.72%


C:\Users\havva\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\havva\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\havva\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [12]:
#SVM

from sklearn.svm import SVC

# SVM için pipeline 
pipeline_svm = Pipeline([
    ('bert', BertTransformer(model, tokenizer)),
    ('clf', SVC())
])

# Hiperparametre aralıkları
param_grid = {
    'clf__C': [0.1],
    'clf__kernel': ['linear'],
}

# GridSearchCV
grid_search_svm = GridSearchCV(pipeline_svm, param_grid, scoring='accuracy', cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42))

# Modeli eğitme ve en iyi parametreler
grid_search_svm.fit(X_train_sampled, y_train_sampled)

# En iyi parametreleri ve sonuçları
print("En iyi parametreler:", grid_search_svm.best_params_)
print("En iyi skor:", grid_search_svm.best_score_)

# Modelin performansı
y_pred_svm = grid_search_svm.predict(X_test_sampled)
print(classification_report(y_test_sampled, y_pred_svm))

# Her bir kombinasyonun performansı
results = grid_search_svm.cv_results_
for mean_score, params in zip(results['mean_test_score'], results['params']):
    print(f"Parametreler: {params}, Doğruluk: {mean_score * 100:.2f}%")


En iyi parametreler: {'clf__C': 0.1, 'clf__kernel': 'linear'}
En iyi skor: 0.751
              precision    recall  f1-score   support

           0       0.68      0.87      0.76       163
           1       0.78      0.76      0.77        82
           2       0.77      0.48      0.59        21
           3       0.74      0.52      0.61        33
           4       1.00      0.50      0.67        12
           5       0.99      0.93      0.96        71
           6       0.81      0.82      0.81        76
           7       0.00      0.00      0.00        12
           8       0.47      0.69      0.56        13
           9       0.88      0.41      0.56        17

    accuracy                           0.76       500
   macro avg       0.71      0.60      0.63       500
weighted avg       0.76      0.76      0.75       500

Parametreler: {'clf__C': 0.1, 'clf__kernel': 'linear'}, Doğruluk: 75.10%


C:\Users\havva\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\havva\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\havva\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [13]:
#RANDOM FOREST

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline

pipeline_rf = Pipeline([
    ('bert', BertTransformer(model, tokenizer)),
    ('clf', RandomForestClassifier())  
])

param_grid_rf = {
    'clf__n_estimators': [150],  
    'clf__max_depth': [30], 
    'clf__min_samples_split': [10] 
}

grid_search_rf = GridSearchCV(pipeline_rf, param_grid_rf, cv=3, verbose=2)
grid_search_rf.fit(X_train_sampled, y_train_sampled)

best_accuracy_rf = grid_search_rf.best_score_
print("En iyi ortalama doğruluk (accuracy) Random Forest:", best_accuracy_rf)

y_pred_rf = grid_search_rf.predict(X_test_sampled)
print(classification_report(y_test_sampled, y_pred_rf))

print("En iyi parametreler Random Forest:", grid_search_rf.best_params_)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] END clf__max_depth=30, clf__min_samples_split=10, clf__n_estimators=150; total time=  15.5s
[CV] END clf__max_depth=30, clf__min_samples_split=10, clf__n_estimators=150; total time=  17.0s
[CV] END clf__max_depth=30, clf__min_samples_split=10, clf__n_estimators=150; total time=  16.7s
En iyi ortalama doğruluk (accuracy) Random Forest: 0.6889980947387954
              precision    recall  f1-score   support

           0       0.63      0.90      0.74       163
           1       0.64      0.78      0.70        82
           2       0.83      0.24      0.37        21
           3       0.69      0.27      0.39        33
           4       1.00      0.08      0.15        12
           5       0.98      0.86      0.92        71
           6       0.78      0.75      0.77        76
           7       0.00      0.00      0.00        12
           8       0.57      0.62      0.59        13
           9       0.00      0.00      

C:\Users\havva\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\havva\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\havva\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [14]:
#rotation forest
from sktime.classification.sklearn import RotationForest
rotation_tree_clf = RotationTreeClassifier()

pipeline_rotation_forest = Pipeline([
    ('bert', BertTransformer(model, tokenizer)),
    ('clf',  RotationForest())
])

param_grid_rotation_forest = {
    'clf__n_estimators': [150],
    'clf__min_group': [5],
    'clf__max_group': [5],
    'clf__remove_proportion': [0.5],
    'clf__n_jobs': [-1],
}


grid_search_rf = GridSearchCV(pipeline_rotation_forest, param_grid_rotation_forest,scoring='accuracy', cv=3, verbose=2)
grid_search_rf.fit(X_train_sampled, y_train_sampled)

best_accuracy_rf = grid_search_rf.best_score_
print("En iyi ortalama doğruluk (accuracy) Rotation Forest:", best_accuracy_rf)

y_pred_rf = grid_search_rf.predict(X_test_sampled)
print(classification_report(y_test_sampled, y_pred_rf))

print("En iyi parametreler Rotation Forest:", grid_search_rf.best_params_)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] END clf__max_group=5, clf__min_group=5, clf__n_estimators=150, clf__n_jobs=-1, clf__remove_proportion=0.5; total time= 2.5min
[CV] END clf__max_group=5, clf__min_group=5, clf__n_estimators=150, clf__n_jobs=-1, clf__remove_proportion=0.5; total time= 2.7min
[CV] END clf__max_group=5, clf__min_group=5, clf__n_estimators=150, clf__n_jobs=-1, clf__remove_proportion=0.5; total time= 2.8min
En iyi ortalama doğruluk (accuracy) Rotation Forest: 0.7107976964030963
              precision    recall  f1-score   support

           0       0.65      0.89      0.75       163
           1       0.68      0.72      0.70        82
           2       0.60      0.29      0.39        21
           3       0.71      0.36      0.48        33
           4       0.75      0.25      0.38        12
           5       1.00      0.87      0.93        71
           6       0.80      0.80      0.80        76
           7       0.50      0.08      0.14

In [15]:
import torch
from torch.utils.data import DataLoader, TensorDataset

pipeline = Pipeline([
    ('bert', BertTransformer(model, tokenizer)),
    ('clf', BaggingClassifier())
])

param_grid = {
    'clf__n_estimators': [100],
    'clf__max_samples': [0.5],
    'clf__max_features': [1.0]
}

grid_search = GridSearchCV(pipeline, param_grid, scoring='accuracy', cv=3, verbose=2)
grid_search.fit(X_train_sampled, y_train_sampled)

best_accuracy = grid_search.best_score_
print("En iyi ortalama doğruluk (accuracy):", best_accuracy)

y_pred = grid_search.predict(X_test_sampled)
print(classification_report(y_test_sampled, y_pred))

print("En iyi parametreler:", grid_search.best_params_)

results = grid_search.cv_results_
for mean_score, params in zip(results['mean_test_score'], results['params']):
    print(f"Parametreler: {params}, Doğruluk: {mean_score * 100:.2f}%")

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] END clf__max_features=1.0, clf__max_samples=0.5, clf__n_estimators=100; total time=  36.2s
[CV] END clf__max_features=1.0, clf__max_samples=0.5, clf__n_estimators=100; total time=  38.7s
[CV] END clf__max_features=1.0, clf__max_samples=0.5, clf__n_estimators=100; total time=  47.7s
En iyi ortalama doğruluk (accuracy): 0.6833974141546241
              precision    recall  f1-score   support

           0       0.63      0.88      0.73       163
           1       0.64      0.73      0.68        82
           2       0.50      0.24      0.32        21
           3       0.69      0.33      0.45        33
           4       0.50      0.08      0.14        12
           5       0.95      0.83      0.89        71
           6       0.76      0.71      0.73        76
           7       0.00      0.00      0.00        12
           8       0.41      0.54      0.47        13
           9       0.00      0.00      0.00        17

 

C:\Users\havva\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\havva\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\havva\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
